In [1]:
# 导入相关库
import typing
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.nn
from torch_geometric.data import Data, DataLoader
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv, GATConv
from model.SplineCNN import SplineCNN

In [2]:
# 定义超参数
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"device : {device}")

from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='./data/Cora', name='Cora')
graph = dataset[0].to(device)

device : cuda


In [3]:
print(graph)
print(f"train num : {graph.train_mask.sum()}")
print(f"val num : {graph.val_mask.sum()}")
print(f"test num : {graph.test_mask.sum()}")
print(f"num classes : {dataset.num_classes}")

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
train num : 140
val num : 500
test num : 1000
num classes : 7


In [4]:
in_channels = dataset.num_features
out_channels = dataset.num_classes

In [5]:
epochs = 256 # 训练轮数
lr = 0.01 # 学习率
val_record_interval = 8

In [6]:
model = SplineCNN(
    in_channels=in_channels,
    out_channels=out_channels,
    ).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = lr, weight_decay=5e-4)
loss_func = nn.NLLLoss()
m = nn.LogSoftmax(dim=1)

In [7]:
loss_x = [e for e in range(epochs)]
loss_record = []
acc_x = [e for e in range(epochs) if e%val_record_interval == 0]
acc_record = []
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    out = model(graph)
    # print(out.shape)
    loss = loss_func(m(out[graph.train_mask]), graph.y[graph.train_mask])
    loss_record.append(loss.item())
    loss.backward()
    optimizer.step()
    if epoch % val_record_interval == 0:
        model.eval()
        _, pred = model(graph).max(dim=1)
        correct = float(pred[graph.val_mask].eq(graph.y[graph.val_mask]).sum().item())
        acc = correct / graph.val_mask.sum().item()
        acc_record.append(acc)

RuntimeError: spline_conv() is missing value for argument 'pseudo'. Declaration: spline_conv(Tensor x, Tensor edge_index, Tensor pseudo, Tensor weight, Tensor kernel_size, Tensor is_open_spline, int degree=1, bool norm=True, Tensor? root_weight=None, Tensor? bias=None) -> Tensor

In [ ]:
import matplotlib.pyplot as plt

plt.subplot(1, 2, 1)
plt.plot(loss_x, loss_record)
plt.title('Loss')
plt.xlabel('epoch')
plt.ylabel('loss')

plt.subplot(1, 2, 2)
plt.plot(acc_x, acc_record)
plt.title('Acc')
plt.xlabel('epoch')
plt.ylabel('acc')

plt.show()

In [ ]:
model.eval()
_, pred = model(graph).max(dim=1)
correct = float(pred[graph.test_mask].eq(graph.y[graph.test_mask]).sum().item())
acc = correct / graph.test_mask.sum().item()
# print(graph.test_mask.sum().item())
print(f"acc : {acc}")